In [1]:
!pip install kaggle
import kaggle


     ---------------------------------------- 0.0/82.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.1 kB ? eta -:--:--
     --------- ---------------------------- 20.5/82.1 kB 108.9 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/82.1 kB 145.2 kB/s eta 0:00:01
     ----------------------- -------------- 51.2/82.1 kB 201.8 kB/s eta 0:00:01
     -------------------------------------  81.9/82.1 kB 305.0 kB/s eta 0:00:01
     -------------------------------------  81.9/82.1 kB 305.0 kB/s eta 0:00:01
     -------------------------------------- 82.1/82.1 kB 219.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105130 sha256=5c33418f23f63c293aacee8fe94a08b4d397aa20394b109ff8cb28c6c903ab82
  Stored in directory: c:\users\dell\appdata\loc

In [2]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders


  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 247kB/s]
100%|##########| 200k/200k [00:00<00:00, 247kB/s]



License(s): CC0-1.0



In [5]:
#extract files
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [6]:
import pandas as pd

In [14]:
#read data and handle null values 
df = pd.read_csv('orders.csv',na_values=['Not Available', 'unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [23]:
#Rename column names, make them lower case and replace spaces with underscore
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.head(5)


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [28]:
#create new columns called discount,sale price and profit
df['discount'] = df['list_price']*df['discount_percent']*0.01
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price'] - df['cost_price']
df.head(5)


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [32]:
#convert order date from object datatype to datetime datatype
df['order_date'] = pd.to_datetime(df['order_date'],format="%Y-%m-%d")
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [34]:
#drop cost price , list price and discount percentage columns from the table
df.drop(columns =['cost_price','list_price','discount_percent'], inplace = True)

In [35]:
#now we will load the date to the sql server 
import sqlalchemy as sal
engine = sal.create_engine('mssql://DESKTOP-3KRHCVA/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [37]:
#load the data into sql server using append option
#df.to_sql('df_orders',con=conn,index=False,if_exists ='replace') this will create coluns with max space wchic will take a lot of space instead use append
df.to_sql('df_orders',con=conn,index=False,if_exists ='append')

38